In [ ]:
import os
import json
import fnmatch
import pandas as pd
import numpy as np
import json
import re
from pprint import pprint

In [2]:
def JSONpath(folder, liga):
    folderlist=os.listdir(folder)
    for f in folderlist:
        if fnmatch.fnmatch(f, f'*{liga}*'):
            filepath=os.path.join(folder,f)
            
    return filepath

In [109]:
def League(liga):
    filename=f'{liga}.csv'
    filepath=os.path.join("stadiums_data",filename)
    df=pd.read_csv(filepath, encoding='utf-8', dtype={"Team": "string", "City": "string", "State": "string", "Stadium Name": "string", 
                                    "Latitude": np.float64, "Longitude": np.float64})
    return df["Team"], df["City"], df["State"], df["Stadium Name"]

In [4]:
def teamfilelist(equipo, path):    
    jsonlista = os.listdir(path)
    teamlista=[]
    for j in jsonlista:
        if fnmatch.fnmatch(j, f'*{equipo}*.json'):
            teamlista.append(j)           
    return teamlista

In [5]:
def natural_sort(s):
    _nsre = re.compile('([0-9]+)')
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)] 

In [32]:
def JSONtoDF(category=False):
    if category:
        inputpath=os.path.join("json_files", "Category_request")
        outputpath=os.path.join("Output", "Category")
    else:
        inputpath=os.path.join("json_files", "General_request")
        outputpath=os.path.join("Output", "General")
        
    liga=input("Input value:\nNFL, NBA, MLB, MLS, NHL: ")
    inputjson=JSONpath(inputpath, liga)
    outputjson=JSONpath(outputpath, liga)
    equipos=League(liga)
    for equipo in equipos:
        equipofiles=teamfilelist(equipo, inputjson)
        equipofiles.sort(key=natural_sort)
        jsonDF=pd.DataFrame()
        for file in equipofiles:
            fpath=os.path.join(inputjson, file)
            with open(fpath, encoding="utf-8") as f:
                jsondata=json.load(f)
            df=pd.DataFrame(jsondata["businesses"])
            jsonDF=pd.concat([jsonDF, df], axis=0)
        jsonDF.reset_index(inplace=True, drop=True)
        fname=f'{equipo}_restaurants.json'
        jsonout=os.path.join(outputjson, fname)
        jsonDF.to_json(jsonout, orient="index")
    

In [33]:
JSONtoDF()

Input value:
NFL, NBA, MLB, MLS, NHL: NHL


In [87]:
def saveJSON(dictio, equipo, outfolder):
    _n=equipo.split()
    if len(_n)==3:
        f_name=f'{_n[0]}_{_n[1]}_{_n[2]}_restaurants.json'
    else:
        f_name=f'{_n[0]}_{_n[1]}_restaurants.json'
    jsonout=os.path.join(outfolder, f_name)
    with open(jsonout, "w", encoding="utf-8") as f:
        json.dump(dictio, f, ensure_ascii=False)
    f.close()

In [110]:
def MergeJSON(category=False):
    if category:
        inputpath=os.path.join("json_files", "Category_request")
        outputpath=os.path.join("Output", "Category")
    else:
        inputpath=os.path.join("json_files", "General_request")
        outputpath=os.path.join("Output", "General")
        
    liga=input("Input value:\nNFL, NBA, MLB, MLS, NHL: ")
    inputjson=JSONpath(inputpath, liga)
    outputjson=JSONpath(outputpath, liga)
    equipos, cities, states, stadiums=League(liga)
    
    for equipo, city, state, stadium in zip(equipos, cities, states, stadiums):
        equipofiles=teamfilelist(equipo, inputjson)
        equipofiles.sort(key=natural_sort)
        team_dict={}
        file0=os.path.join(inputjson, equipofiles[0])
        allbusiness=[]
        with open(file0, encoding="utf-8") as f:
            file0data=json.load(f)
            total=file0data["total"]
        for file in equipofiles:
            fpath=os.path.join(inputjson, file)
            with open(fpath, encoding="utf8") as f:
                jsondata=json.load(f)
                businessdata=jsondata['businesses']
            allbusiness.extend(businessdata)
        team_dict['league']=liga
        team_dict['team']=equipo
        team_dict['stadium']=stadium
        team_dict['city']=city
        team_dict['state']=state
        team_dict['businesses']=allbusiness
        team_dict['absolute total']=total
        if total>=1000:
            team_dict['total']=1000
        else:
            team_dict['total']=total
        saveJSON(team_dict, equipo, outputjson)

In [111]:
MergeJSON()

Input value:
NFL, NBA, MLB, MLS, NHL: NHL
